# Fine Tuning With Flan T5 XL

Author : Sivaprasad Puthumadathil Rameshan Nair

Flan-T5 XL Model

- 3 Billion Parameters
- Model Type: Language model
- Architecture: Flan-T5 XL is based on the T5 (Text-To-Text Transfer Transformer) architecture developed by Google, which uses a transformer-based approach that is highly effective for various natural language processing (NLP) tasks.
- Performance: The model excels in a wide range of tasks, including translation, summarization, question answering, and text generation.
Instruction Fine-Tuning: Flan-T5 XL has been fine-tuned with instruction-based data, showing significant improvements in benchmarks over models that were not fine-tuned in this manner.
- Human-Like Text Generation: It has demonstrated exceptional performance in tasks that require both understanding and generating human-like text, making it highly suitable for applications that require nuanced and coherent language generation.

## 1. Story generation before fine tuning

In [19]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl").to(device)  # Ensure the model is on the same device

# Function to generate text
def generate_story_with_keywords(keywords, emotion, userpref, max_length=150):
    # Create the prompt for the story generation
    prompt = (
        f"Generate a story that evokes a {emotion} emotion. The story should feature a "
        f"{keywords[0]}, a {keywords[1]}, and a {keywords[2]}. "
        f"Additionally, incorporate elements of {userpref} to enhance the narrative. "
        f"Ensure the {userpref} aspects are seamlessly integrated and contribute to the overall {emotion} tone of the story."
    )

    # Encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)  # Ensure the inputs are on the same device

    # Generate the story
    start_time = time.time()
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        temperature=0.7,  # Control the randomness of predictions
        top_k=50  # Limit the sampling pool to top_k tokens
    )
    end_time = time.time()

    # Decode the generated text
    story = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Time taken to generate the story: {end_time - start_time:.2f} seconds")
    return story

# Define the keywords, emotion, and user preferences
keywords = ["dog", "sun", "beach"]
emotion = "happy"
userpref = "history"

# Generate the story
story = generate_story_with_keywords(keywords, emotion, userpref, max_length=512)

# Function to save the story to a file
def save_story_to_file(story, filename='/content/sample_data/generated_story_flan-t5-xl.txt'):
    with open(filename, 'w') as file:
        file.write(story)

# Save the generated story to a file
save_story_to_file(story)

# Print the generated story
print(story)


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Time taken to generate the story: 1.86 seconds
The dog was happy to be on the beach. He was laying in the sun and enjoying the warm weather. The sun was shining brightly and the dog loved the warmth.


## 2. Dataset Preprocessing

In [1]:
import pandas as pd

# Read the CSV file while skipping problematic lines
processed_data = pd.read_csv('/content/sample_data/stories_with_features_with_genre.csv', on_bad_lines='warn')
processed_data.shape


(1000, 9)

In [2]:
processed_data.head()

,id,story,genre,characters,objects,locations,vehicles,professions,emotions
0,457580,"In the year 2250, Earth had made significant s...",Science Fiction,"scientist, star, Shadowbeast, Reynolds, UEG, e...","ship, game","spacecraft, fortress, field, moon Europa",NaN,inventor,"despair, hope, excitement"
1,297904,"In a land far away, where the sun shone bright...",Fantasy,"the Shadow Beast's, Thorn, Eldoria, sorcerer, ...","Sword, puzzle, scroll, sword","the Sword of Eldoria, The Sword of Eldoria, br...",NaN,adventurer,"determination, excitement"
2,620436,"Once upon a time, in a small, tranquil town ca...",Mystery,"detective, Thomas, Johnathan, Whispering Shado...",NaN,"valley, town, warehouse, city, square",NaN,NaN,"shock, love, hope, determination, gratitude, g..."
3,634687,"Once upon a time in the 16th century, a small ...",Historical Adventure,"William, Elias, the Emerald Amulet, Blackwood,...",key,"temple, town, trail, village",NaN,NaN,"hope, determination, gratitude"
4,513427,In the sun-drenched coastal city of St. August...,Thriller,"Alex, Florida, Katie, Sarah, Thomas, artist, P...","computer, map, game","bar, city, ocean, Laboratory",NaN,lawyer,"despair, hope, determination"


In [3]:
processed_data.columns

Index(['id', 'story', 'genre', 'characters', 'objects', 'locations',
       'vehicles', 'professions', 'emotions'],
      dtype='object')

In [4]:
processed_data.isna().sum()

id               0
story            0
genre            0
characters       1
objects        344
locations       47
vehicles       976
professions    728
emotions       133
dtype: int64

In [5]:
df = processed_data.drop(columns=['vehicles'])
df.columns

Index(['id', 'story', 'genre', 'characters', 'objects', 'locations',
       'professions', 'emotions'],
      dtype='object')

In [6]:
# Handle NaN values: Fill NaN values with a default string
df.fillna('Unknown', inplace=True)
df.isna().sum()

id             0
story          0
genre          0
characters     0
objects        0
locations      0
professions    0
emotions       0
dtype: int64

In [7]:
df.head()

,id,story,genre,characters,objects,locations,professions,emotions
0,457580,"In the year 2250, Earth had made significant s...",Science Fiction,"scientist, star, Shadowbeast, Reynolds, UEG, e...","ship, game","spacecraft, fortress, field, moon Europa",inventor,"despair, hope, excitement"
1,297904,"In a land far away, where the sun shone bright...",Fantasy,"the Shadow Beast's, Thorn, Eldoria, sorcerer, ...","Sword, puzzle, scroll, sword","the Sword of Eldoria, The Sword of Eldoria, br...",adventurer,"determination, excitement"
2,620436,"Once upon a time, in a small, tranquil town ca...",Mystery,"detective, Thomas, Johnathan, Whispering Shado...",Unknown,"valley, town, warehouse, city, square",Unknown,"shock, love, hope, determination, gratitude, g..."
3,634687,"Once upon a time in the 16th century, a small ...",Historical Adventure,"William, Elias, the Emerald Amulet, Blackwood,...",key,"temple, town, trail, village",Unknown,"hope, determination, gratitude"
4,513427,In the sun-drenched coastal city of St. August...,Thriller,"Alex, Florida, Katie, Sarah, Thomas, artist, P...","computer, map, game","bar, city, ocean, Laboratory",lawyer,"despair, hope, determination"


## 3. Loading the data for fine tuning

let's concatenates various columns from the dataset into a single text string. This string is intended to be used as input for fine-tuning the model. The idea is to create a rich and informative prompt that includes multiple aspects of the story, such as characters, objects, locations, professions, and emotions.

How It Works:

- data['story']: Contains the main story text.
- data['characters']: Contains characters mentioned in the story.
- data['objects']: Contains objects referenced in the story.
- data['locations']: Contains locations where the story takes place.
- data['professions']: Contains professions of characters in the story.
- data['emotions']: Contains emotions depicted in the story.

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [8]:
from datasets import Dataset
from transformers import AutoTokenizer

# Concatenate input features into a single input string, without the vehicle column
df['input_text'] = df.apply(lambda row: f"Genre: {row['genre']} Characters: {row['characters']} Objects: {row['objects']} Locations: {row['locations']} Professions: {row['professions']} Emotions: {row['emotions']}", axis=1)
df['target_text'] = df['story']

# Convert the DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(df[['input_text', 'target_text']])


## 4. Tokenising the data

Tokenize the text data to prepare it for model training.

Tokenizing the data is a crucial step in preparing text for model training, especially for transformer models like T5

Why Tokenize the Data?

- Converting Text to Numerical Format: Machine learning models, particularly neural networks, require numerical input. Tokenization converts text into a sequence of numbers (token IDs) that the model can process.
- Handling Vocabulary: Tokenization breaks down text into smaller units (tokens), such as words or subwords, and maps each token to a unique ID in the model's vocabulary. This helps the model understand and generate text.
- Managing Input Length: Tokenization ensures that text inputs are appropriately truncated or padded to a fixed length. This uniformity is essential for batch processing in model training.
- Preserving Meaning: Advanced tokenizers (like the one used for T5) often use subword units, which helps in handling out-of-vocabulary words and preserving the semantic meaning of the text.

What Does Tokenization Involve?

- Splitting Text into Tokens: The text is split into smaller units (tokens), which can be words, subwords, or characters.
- Mapping Tokens to IDs: Each token is mapped to a unique ID in the model’s vocabulary.
- Truncating or Padding Sequences: The tokenized sequences are truncated to a maximum length if they are too long, or padded with special tokens if they are too short. This ensures all sequences in a batch have the same length.

In [9]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')

# Tokenize the dataset with padding
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples['target_text'], max_length=512, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
# Visualize a few samples of the tokenized data
for i in range(3):  # Change the range to see more or fewer samples
    print(f"Sample {i+1}:")
    print("Tokenized Input IDs:", tokenized_dataset[i]['input_ids'])
    print("Tokenized Labels IDs:", tokenized_dataset[i]['labels'])
    print("")

Sample 1:
Tokenized Input IDs: [5945, 60, 10, 2854, 24525, 20087, 7, 10, 17901, 6, 2213, 6, 18136, 115, 11535, 6, 27815, 6, 412, 8579, 6, 9739, 6, 30059, 6, 10498, 6, 11856, 6, 8, 638, 7, 3113, 391, 99, 17, 19746, 2661, 7038, 6, 12202, 27815, 6, 13622, 6, 24308, 6, 19553, 6, 3, 4256, 6, 8114, 6, 4030, 6, 30486, 6, 1079, 96, 683, 4365, 121, 27815, 6, 17687, 6, 13962, 6, 736, 13240, 10498, 6, 37, 907, 4030, 3141, 6, 11566, 120, 29, 96, 427, 162, 1686, 12202, 6, 205, 13223, 6, 160, 32, 3, 17057, 7, 10, 4383, 6, 467, 10450, 7, 10, 628, 6696, 6, 21, 9746, 6, 1057, 6, 8114, 5578, 749, 17585, 7, 10, 21244, 262, 7259, 7, 10, 25802, 6, 897, 6, 10147, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 5. Fine tuning the model


In [11]:
!pip uninstall transformers torch datasets accelerate -y


Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121
Found existing installation: datasets 2.20.0
Uninstalling datasets-2.20.0:
  Successfully uninstalled datasets-2.20.0


In [12]:
!pip install transformers==4.27.4 torch datasets accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached

In [11]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load the pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-xl')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results/flan-t5-xl',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduce batch size
    per_device_eval_batch_size=1,   # Reduce batch size
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps=10_000,
    logging_dir='./logs',
    gradient_accumulation_steps=8,  # Increase gradient accumulation steps
    fp16=True,  # Enable mixed precision training
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You might want to create a separate validation dataset
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,nan
2,No log,nan
3,No log,nan


TrainOutput(global_step=375, training_loss=0.0, metrics={'train_runtime': 1078.0135, 'train_samples_per_second': 2.783, 'train_steps_per_second': 0.348, 'total_flos': 2.5656947638272e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
# to clean up the memory

import os
os._exit(00)  # This will restart the runtime


In [12]:
import torch
torch.cuda.empty_cache()


In [12]:
model.save_pretrained('/content/sample_data/fine-tuned-t5-xl')
tokenizer.save_pretrained('/content/sample_data/fine-tuned-t5-xl')

('/content/sample_data/fine-tuned-t5-xl/tokenizer_config.json',
 '/content/sample_data/fine-tuned-t5-xl/special_tokens_map.json',
 '/content/sample_data/fine-tuned-t5-xl/spiece.model',
 '/content/sample_data/fine-tuned-t5-xl/added_tokens.json',
 '/content/sample_data/fine-tuned-t5-xl/tokenizer.json')

In [13]:
import shutil

shutil.make_archive('/content/sample_data/fine-tuned-t5-xl', 'zip', '/content/sample_data/fine-tuned-t5-xl')


'/content/sample_data/fine-tuned-t5-xl.zip'

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
# Copy the compressed file to Google Drive
shutil.copy('/content/sample_data/fine-tuned-t5.zip', '/content/drive/MyDrive/fine-tuned-t5.zip')


'/content/drive/MyDrive/fine-tuned-t5.zip'

## 6. Generate Stories with the Fine-Tuned Model

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load the fine-tuned model and tokenizer
model_path = '/content/sample_data/fine-tuned-t5-xl'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Create a pipeline for text generation
text2text_generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Define the prompt
keywords = ["dog", "sun", "beach"]
emotion = "happy"
userpref = "history"

prompt = (
    f"Generate a story that evokes a {emotion} emotion. The story should feature a "
    f"{keywords[0]}, a {keywords[1]}, and a {keywords[2]}. "
    f"Additionally, incorporate elements of {userpref} to enhance the narrative. "
    f"Ensure the {userpref} aspects are seamlessly integrated and contribute to the overall {emotion} tone of the story."
)

# Generate the story
generated_story = text2text_generator(prompt, max_length=512, do_sample=True, top_p=0.95, num_return_sequences=1)

# Print the generated story
print(generated_story[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The dog wanted to be outside and had a good day on the beach. So he went out in the sun with a beach toy, and spent all day on the sand.


In [21]:
# Extract the generated text
story_text = generated_story[0]['generated_text']

# Save the generated story to a text file
output_file_path = '/content/sample_data/generated_story_flan_t5_XL_finetuned.txt'
with open(output_file_path, 'w') as file:
    file.write(story_text)

print(f"Generated story saved to {output_file_path}")

Generated story saved to /content/sample_data/generated_story_flan_t5_XL_finetuned.txt


## 7. Performance

### 7.1 Qualitative Evaluation

#### **Human Evaluation**

- Content Relevance: The story should feature a dog, the sun, and the beach.

  - Check: The story prominently features a dog, mentions the sun, and describes the beach.

- Emotion Elicitation: The story should evoke a happy emotion.

  - Check: The story conveys a sense of happiness through the dog's enjoyment of the day on the beach.

- Incorporation of User Preference (History): The story should include historical elements seamlessly.

  - Check: The story does not include any historical elements.

- Coherence and Fluency: The story should be coherent and fluent.

  - Check: The story is coherent and fluent with no issues affecting readability.


### 7.2 Quantitative Evaluation

####  **BLEU and ROUGE Scores**

In [25]:
from datasets import load_metric

# Load the BLEU and ROUGE metrics
bleu = load_metric('bleu')
rouge = load_metric('rouge')

# Load the generated stories from the files
with open('/content/sample_data/generated_story_flan-t5-xl.txt', 'r', encoding='utf-8') as file:
    story_before_fine_tuning = file.read()

with open('/content/sample_data/generated_story_flan_t5_XL_finetuned.txt', 'r', encoding='utf-8') as file:
    story_after_fine_tuning = file.read()

# Load the reference story from the file
with open('/content/sample_data/reference_story.txt', 'r', encoding='utf-8') as file:
    reference_story = file.read()

# Tokenize the generated and reference stories
generated_tokens_before = [story_before_fine_tuning.split()]
generated_tokens_after = [story_after_fine_tuning.split()]
reference_tokens = [[reference_story.split()]]  # Reference story tokenized

# Compute BLEU scores
bleu_result_before = bleu.compute(predictions=generated_tokens_before, references=reference_tokens)
bleu_result_after = bleu.compute(predictions=generated_tokens_after, references=reference_tokens)

print("BLEU Score Before Fine-Tuning:", bleu_result_before['bleu'])
print("BLEU Score After Fine-Tuning:", bleu_result_after['bleu'])

# Compute ROUGE scores
rouge_result_before = rouge.compute(predictions=[story_before_fine_tuning], references=[reference_story])
rouge_result_after = rouge.compute(predictions=[story_after_fine_tuning], references=[reference_story])

print("ROUGE Scores Before Fine-Tuning:", rouge_result_before)
print("ROUGE Scores After Fine-Tuning:", rouge_result_after)


BLEU Score Before Fine-Tuning: 0.0
BLEU Score After Fine-Tuning: 0.0
ROUGE Scores Before Fine-Tuning: {'rouge1': AggregateScore(low=Score(precision=0.7741935483870968, recall=0.08304498269896193, fmeasure=0.15), mid=Score(precision=0.7741935483870968, recall=0.08304498269896193, fmeasure=0.15), high=Score(precision=0.7741935483870968, recall=0.08304498269896193, fmeasure=0.15)), 'rouge2': AggregateScore(low=Score(precision=0.16666666666666666, recall=0.017361111111111112, fmeasure=0.031446540880503145), mid=Score(precision=0.16666666666666666, recall=0.017361111111111112, fmeasure=0.031446540880503145), high=Score(precision=0.16666666666666666, recall=0.017361111111111112, fmeasure=0.031446540880503145)), 'rougeL': AggregateScore(low=Score(precision=0.5161290322580645, recall=0.05536332179930796, fmeasure=0.1), mid=Score(precision=0.5161290322580645, recall=0.05536332179930796, fmeasure=0.1), high=Score(precision=0.5161290322580645, recall=0.05536332179930796, fmeasure=0.1)), 'rougeLsu

In [21]:
!pip install rouge-score
!pip install datasets


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=01afa78ffb97da23857282f299af0a023927da3f055651e70df35b0fd7af8fa0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


#### **Perplexity**

In [26]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the tokenizer and model
model_path = '/content/sample_data/fine-tuned-t5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Load the generated stories from the files
with open('/content/sample_data/generated_story_flan-t5-xl.txt', 'r', encoding='utf-8') as file:
    story_before_fine_tuning = file.read()

with open('/content/sample_data/generated_story_flan_t5_XL_finetuned.txt', 'r', encoding='utf-8') as file:
    story_after_fine_tuning = file.read()

# Tokenize the input texts
inputs_before = tokenizer(story_before_fine_tuning, return_tensors='pt')
inputs_after = tokenizer(story_after_fine_tuning, return_tensors='pt')

# Move tensors to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
inputs_before = {key: value.to(device) for key, value in inputs_before.items()}
inputs_after = {key: value.to(device) for key, value in inputs_after.items()}

# Compute loss for before fine-tuning
with torch.no_grad():
    outputs_before = model(**inputs_before, labels=inputs_before["input_ids"])
    loss_before = outputs_before.loss
    perplexity_before = torch.exp(loss_before)

# Compute loss for after fine-tuning
with torch.no_grad():
    outputs_after = model(**inputs_after, labels=inputs_after["input_ids"])
    loss_after = outputs_after.loss
    perplexity_after = torch.exp(loss_after)

print("Perplexity Before Fine-Tuning:", perplexity_before.item())
print("Perplexity After Fine-Tuning:", perplexity_after.item())


Perplexity Before Fine-Tuning: 1.931144118309021
Perplexity After Fine-Tuning: 1.298383116722107


- The decrease in perplexity after fine-tuning demonstrates an improvement in the model's ability to generate text that is similar to the training data.

- A lower perplexity value after fine-tuning indicates that the model has become more adept at capturing the patterns and structures in the text, leading to more accurate predictions.